<a href="https://colab.research.google.com/github/skyler-ud/MLTSA_SLindsey/blob/main/labs/Copy_of_TimeSeriesClassificationWithKerasTransformers_ClassCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import keras
from keras import layers

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

input_shape = x_train.shape[1:]

In [ ]:
x_train.shape

(3601, 500, 1)

In [ ]:
def train_and_evaluate(model, Xtrn, Xtst, Ytrn, Ytst):
    cb_early = keras.callbacks.EarlyStopping(
        patience=10, restore_best_weights=True
    )
    model.fit(
        Xtrn,
        Ytrn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[cb_early],
    )
    loss_tst, acc_tst = model.evaluate(Xtst, Ytst,verbose=1)
    print(f"Loss calculated on the testing set: {loss_tst:.4f}")
    print(f"Accuracy calculated on the testing set: {acc_tst:.4f}")
    return None

In [ ]:
def transformer_block(
    x,
    input_shape,
    heads=4,
    key_dim=256,
    encoder_proj_dim=4,
    dropout_attention=0.25,
    dropout_projection=0.25,
):
    x0 = layers.MultiHeadAttention(num_heads=heads,key_dim=key_dim)(x,x)
    x0 = layers.Dropout(dropout_attention)(x0)
    x0 = layers.Add()([x,x0])
    x0 = layers.LayerNormalization()(x0)

    x1 = layers.Conv1D(filters=encoder_proj_dim,kernel_size=1,activation='relu')(x0)
    x1 = layers.Conv1D(filters=input_shape[1],kernel_size=1,activation='relu')(x1)
    x1 = layers.Dropout(dropout_projection)(x1)
    x1 = layers.Add()([x0,x1])
    x1 = layers.LayerNormalization()(x1)

    return x1


def build_transformer_model(
    input_shape,
    encoder_blocks=0,
    feed_forward_units=[128]
):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    for _ in range(encoder_blocks):
        x = transformer_block(x,input_shape)

    x = layers.GlobalMaxPooling1D(data_format="channels_first")(x)
    for n in feed_forward_units:
        x = layers.Dense(n, activation="relu")(x)
        x = layers.Dropout(.4)(x)

    outputs = layers.Dense(2,activation='softmax')(x)
    model = keras.Model(inputs,outputs)

    return model


transformer_model = build_transformer_model(input_shape)
transformer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
transformer_model.summary()

train_and_evaluate(transformer_model, x_train, x_test, y_train, y_test)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 500, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 500)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,386 (251.51 KB)

 Trainable params: 64,386 (251.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 1.1753 - sparse_categorical_accuracy: 0.5149 - val_loss: 0.7508 - val_sparse_categorical_accuracy: 0.5742
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8815 - sparse_categorical_accuracy: 0.5692 - val_loss: 0.6556 - val_sparse_categorical_accuracy: 0.6380
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7885 - sparse_categorical_accuracy: 0.5896 - val_loss: 0.6232 - val_sparse_categorical_accuracy: 0.6602
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6999 - sparse_categorical_accuracy: 0.6262 - val_loss: 0.5997 - val_sparse_categorical_accuracy: 0.6907
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6566 - sparse_categorical_accuracy: 0.6532 - val_loss: 0.5801 - val_sparse_categorical_accuracy: 0.6963
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6575 - sparse_categorical_accuracy: 0.6705 - val_loss: 0.5717 - val_sparse_categorical_accuracy: 0.7143
Epoch 7/1

In [ ]:
def build_ff_model(
    input_shape,
    feed_forward_units=[128]
):
    inputs = ...
    x = inputs

    x = ...
    for ... in ...:
        x = ...
        x = ...

    outputs = ...
    model = keras.Model(..., ...)

    return model


input_shape = x_train.shape[1:]
ff_model = build_ff_model(input_shape)
ff_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=...),
    metrics=["sparse_categorical_accuracy"],
)
ff_model.summary()

train_and_evaluate(ff_model, x_train, x_test, y_train, y_test)